In [1]:
from utils.imports import *

Using TensorFlow backend.


## 训练模型

In [2]:
src = PATH['model_train']
model_paths = PATH['model_paths']

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return ((2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1))

def unet_model(dropout_rate,learn_rate, width):
    inputs = Input((1, 512, 512))
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="elu")(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="elu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="elu")(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="elu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="elu")(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="elu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="elu")(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="elu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="elu")(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="elu")(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = SpatialDropout2D(dropout_rate)(up6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="elu")(conv6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="elu")(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = SpatialDropout2D(dropout_rate)(up7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="elu")(conv7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="elu")(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = SpatialDropout2D(dropout_rate)(up8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="elu")(conv8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="elu")(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = SpatialDropout2D(dropout_rate)(up9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="elu")(conv9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="elu")(conv9)
    conv10 = Convolution2D(1, (1, 1), activation="sigmoid")(conv9)

    model = Model(input=inputs, output=conv10)
    #model.summary()
    model.compile(optimizer=Adam(lr=learn_rate), loss=dice_coef_loss, metrics=[dice_coef])
    #model.compile(optimizer=SGD(lr=learn_rate, momentum=0.9, nesterov=True), loss=dice_coef_loss, metrics=[dice_coef])
    
    #plot_model(model, to_file='model1.png',show_shapes=True)
    return model


def unet_fit(name, check_name = None):
    data_gen_args = dict(rotation_range=90.,   
                     width_shift_range=0.3,  
                     height_shift_range=0.3,   
                     horizontal_flip=True,   
                     vertical_flip=True,  
                     )
    from keras.preprocessing.image import ImageDataGenerator
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    
    image_generator = image_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['lung'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1)

    mask_generator = mask_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['nodule'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1) 
    # combine generators into one which yields image and masks
    train_generator = itertools.izip(image_generator, mask_generator)   
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 40, 
                                   verbose = 1),
    ModelCheckpoint(model_paths + '{}.h5'.format(name), 
                        monitor='val_loss', 
                        verbose = 0, save_best_only = True)]
    
    if check_name is not None:
        check_model = model_paths + '{}.h5'.format(check_name)
        model = load_model(check_model, 
                           custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    else:
        model = unet_model(dropout_rate = 0.35, learn_rate = 1e-5, width = 32)
    model.fit_generator(
        train_generator,
        epochs=300,
        verbose =1, 
        callbacks = callbacks,
        steps_per_epoch=256,
        validation_data = train_generator,
        nb_val_samples = 48)
    return

In [ ]:
#unet_fit('final_fenge_170626')
unet_fit('final_fenge_170626','final_fenge_170626')

Found 3711 images belonging to 1 classes.
Found 3711 images belonging to 1 classes.
Epoch 1/300
256/256 [==============================] - 96s - loss: -1.1876 - dice_coef: 1.1876 - val_loss: -0.9817 - val_dice_coef: 0.9817
Epoch 2/300
256/256 [==============================] - 92s - loss: -1.2123 - dice_coef: 1.2123 - val_loss: -0.9181 - val_dice_coef: 0.9181
Epoch 3/300
256/256 [==============================] - 94s - loss: -1.1728 - dice_coef: 1.1728 - val_loss: -1.0871 - val_dice_coef: 1.0871
Epoch 4/300
256/256 [==============================] - 94s - loss: -1.2254 - dice_coef: 1.2254 - val_loss: -1.3041 - val_dice_coef: 1.3041
Epoch 5/300
256/256 [==============================] - 92s - loss: -1.1811 - dice_coef: 1.1811 - val_loss: -0.8569 - val_dice_coef: 0.8569
Epoch 6/300
256/256 [==============================] - 92s - loss: -1.2306 - dice_coef: 1.2306 - val_loss: -0.9389 - val_dice_coef: 0.9389
Epoch 7/300
256/256 [==============================] - 92s - loss: -1.2613 - dice_

256/256 [==============================] - 92s - loss: -1.4980 - dice_coef: 1.4980 - val_loss: -1.3552 - val_dice_coef: 1.3552
Epoch 59/300
256/256 [==============================] - 92s - loss: -1.4413 - dice_coef: 1.4413 - val_loss: -1.3193 - val_dice_coef: 1.3193
Epoch 60/300
256/256 [==============================] - 92s - loss: -1.3510 - dice_coef: 1.3510 - val_loss: -1.3648 - val_dice_coef: 1.3648
Epoch 61/300
256/256 [==============================] - 92s - loss: -1.4303 - dice_coef: 1.4303 - val_loss: -1.1269 - val_dice_coef: 1.1269
Epoch 62/300
256/256 [==============================] - 92s - loss: -1.4835 - dice_coef: 1.4835 - val_loss: -1.3168 - val_dice_coef: 1.3168
Epoch 63/300
256/256 [==============================] - 92s - loss: -1.4570 - dice_coef: 1.4570 - val_loss: -1.5175 - val_dice_coef: 1.5175
Epoch 64/300
256/256 [==============================] - 92s - loss: -1.4738 - dice_coef: 1.4738 - val_loss: -1.4537 - val_dice_coef: 1.4537
Epoch 65/300
256/256 [===========